# Machine Learning

Andrew Esch, Evan Lee, Collin Stratton

Dr. Isac Artzi

CST-425

4/17/2022

# Introduction
The purpose of this assignment is to synthesize the complex aspects of machine learning. Specifically, this project will create an image model that will be able to predict the type of food an image of a food item is, and then pass in an image of a dog to see what type of food the dog is. This is a fun way to implement machine learning in a unique way to showcase the skills learned in this course. 

# Code
The outline of the code is as follows: First, the appropriate libraries are install and the images are loaded. Next, the images are labeled and categorized for preprocessing. Then, the images are preprocessed and the model is trained. Finally, the model is used to predict what type of food an image of a dog is.  

## Setup

In [1]:
# import libraries
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os.path
import numpy as np
import pandas as pd

from pathlib import Path

import requests
from PIL import Image
from io import BytesIO
import cv2

## Get Images and Label Data

In [2]:
# import images
# image_dir = Path('C:/Users/Drew/OneDrive/Pictures/ml-food/images')
image_dir = Path('/Users/collinstratton/Documents/archive/images')
images_paths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], images_paths))

images_paths = pd.Series(images_paths, name='Image_Path').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([images_paths, labels], axis=1)

In [3]:
# split images into categories
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query('Label == @category')
    category_samples.append(category_slice.sample(300, random_state=1))

# group images into tables and check the number of images by category
images_samples = pd.concat(category_samples, axis=0)
images_samples['Label'].value_counts()

chocolate_cake           300
gyoza                    300
french_toast             300
lobster_roll_sandwich    300
lobster_bisque           300
                        ... 
paella                   300
carrot_cake              300
peking_duck              300
red_velvet_cake          300
caprese_salad            300
Name: Label, Length: 101, dtype: int64

## Preprocess Images

In [4]:
# Testing and training
train_data, test_data = train_test_split(images_samples, train_size=0.7, shuffle=True, random_state=1)

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)

# freeze the weights of the network to maintain transfer learning
pretrained_model.trainable = False

inputs = pretrained_model.input

# create two layers of our own
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# create output layer with the 101 classes
output_layer = tf.keras.layers.Dense(101, activation='softmax')(x)

# unite the original model with the new layers
model = tf.keras.Model(inputs, output_layer)

print(model.summary())

Found 16968 validated image filenames belonging to 101 classes.
Found 16968 validated image filenames belonging to 101 classes.
Found 9090 validated image filenames belonging to 101 classes.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Co

## Train Model

In [8]:
# Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

# Training Results - Prediction Accuracy
results = model.evaluate(test_images, verbose=1)
print("Precision: {:.2f}%".format(results[1] * 100))

Epoch 1/50
531/531 [==============================] - 626s 1s/step - loss: 2.8741 - accuracy: 0.3091 - val_loss: 1.9647 - val_accuracy: 0.4920
Epoch 2/50
531/531 [==============================] - 633s 1s/step - loss: 1.9203 - accuracy: 0.5018 - val_loss: 1.5577 - val_accuracy: 0.5878
Epoch 3/50
531/531 [==============================] - 2485s 5s/step - loss: 1.6217 - accuracy: 0.5695 - val_loss: 1.3397 - val_accuracy: 0.6458
Epoch 4/50
531/531 [==============================] - 3135s 6s/step - loss: 1.3840 - accuracy: 0.6254 - val_loss: 1.1026 - val_accuracy: 0.7063
Epoch 5/50
531/531 [==============================] - 3582s 7s/step - loss: 1.1983 - accuracy: 0.6712 - val_loss: 0.9571 - val_accuracy: 0.7373
Epoch 6/50
531/531 [==============================] - 3621s 7s/step - loss: 1.0226 - accuracy: 0.7124 - val_loss: 0.8155 - val_accuracy: 0.7762
Epoch 7/50
531/531 [==============================] - 3015s 6s/step - loss: 0.8705 - accuracy: 0.7514 - val_loss: 0.6777 - val_accuracy: 0

KeyboardInterrupt: 

## Image Prediction

In [1]:
# find what food a dog image looks like
# 'C:/GCU Academics/Junior Year/Second Semester/CST-425/CLC Shared GitHub Repository/Cloned Repository/MachineLearning/Daisy Pic.png'
#'/Users/collinstratton/Documents/archive/dog_images/17DOGS-mobileMasterAt3x-v2.jpg'
img = Image.open('/Users/collinstratton/Documents/archive/dog_images/17DOGS-mobileMasterAt3x-v2.jpg')
img.show()

img = np.array(img).astype(float) / 255

img = cv2.resize(img, (224, 224))
predict = model.predict(img.reshape(-1, 224, 224, 3))

NameError: name 'Image' is not defined

In [12]:
# output the result
key_list = list(test_images.class_indices.keys())
value_list = list(test_images.class_indices.values())

position = value_list.index(np.argmax(predict))
print(key_list[position])

macaroni_and_cheese


# Conclusion
This project was a fun way to implement machine learning in a unique way to showcase the skills learned in this course. The code is very simple and the results are very interesting. Even those the model was trained to be able to predict what type of food is in an image, it is versatile enough to be able to predict what type of food a dog is. The model got up to 80% accuracy on the food set, so we can only assume its high accuracy in predicting what type of food a dog is. 

# Resources
https://www.kaggle.com/datasets/kmader/food41

https://www.kaggle.com/code/sergiogarridomerino/clasificaci-n-de-comida

https://images.cv
